## accountSnapshot - Overview

**accountSnapshot** is at time of writing the second most popular API query called by Cato customers, and the number 1 query called by Cato resellers and MSPs. The main use cases for accountSnapshot are:
* Retrieving site and user connectivity status, especially site and tunnel connectivity.
* Retrieving site and user configuration.
* Retrieving historical connection data, especially user device activity.

It is one of the simplest API queries to call, requiring only the account ID, and an ideal candidate as a minimalistic example and "ping" to confirm that an API ID/key combination are working. It is also one of the largest queries in terms of the number of fields which can be included in the response.

### Minimal example

A simple query to "ping" the API and return the account ID looks like this:


In [ ]:
query = """
query accountSnapshot($accountID:ID!) {
    accountSnapshot(accountID:$accountID) {
        id
    }
}
"""

Assuming that the account ID and API key are preloaded as environment variables and using our helper module (see the **Getting Started** notebook if this is unclear):

In [ ]:
import os
from cato import API
C = API(os.environ["CATO_API_KEY"])
variables = {"accountID": os.environ["CATO_ACCOUNT_ID"]}
success, result = C.send("accountSnapshot", variables, query)
print(result)

### Using accountSnapshot to return site connectivity status

One of the most common accountSnapshot use cases is to retrieve site connectivity status:

In [ ]:
query = """
query accountSnapshot($accountID:ID!) {
    accountSnapshot(accountID:$accountID) {
        sites {
            id
            info {
                name
            }
            connectivityStatus
        }
    }
}
"""
success, result = C.send("accountSnapshot", variables, query)
print(result)

### Filtering to only see data for particular sites ###
By default **accountSnapshot** will return data for all configured sites in the account, but sometimes you might only want to request data for a subset of sites. This can be achieved using the siteIDs parameter at the site level. How do you know which IDs to use? There are several ways to map site names to site IDs, with the two most common being:
* Use the [entityLookup](https://github.com/catonetworks/data-analytics/blob/main/notebooks/Entities%2C%20IDs%20and%20entityLookup.ipynb) query with type=site to fetch a list of site entities.
* Use an accountSnapshot query similar to the one above to fetch sites with ID and name fields.

Having done this, you can use this in the siteIDs input at the site level. If we're interested in two particular sites with IDs 9668 and 78508, we can update the query with the siteIDs variable and add the input to the query variables, like this:

In [ ]:
query = """query accountSnapshot($accountID:ID! $siteIDs:[ID!]) {
    accountSnapshot(accountID:$accountID) {
        id
    		sites (siteIDs:$siteIDs) {
          id
          info {
            name
          }
          connectivityStatus
        }
    }
}"""
variables["siteIDs"] = [9668, 78508]
success, result = C.send("accountSnapshot", variables, query)
print(result)

### Users and accountSnapshot ###
User connectivity status, connection history and device data can be retrieved with accountSnapshot in much the same way as site information, but with one key difference: by default, accountSnapshot will only return data for SDP users who are currently connected. For some use cases this is sufficient, but for others (such as wanting to know the date of last user connection) you probably want to see data for disconnected users as well. This is easy to do using the same principle as for sites - obtain a list of user IDs and submit them at the user level in the userIDs parameter.